In [3]:
from DistanceHelper.DistanceDetector import detectPeople
from DistanceHelper.DistanceDetector import MIN_CONF
from DistanceHelper.DistanceDetector import MIN_DISTANCE
from DistanceHelper.DistanceDetector import NMS_THRESH

from scipy.spatial import distance as dist
import numpy as np
import argparse
import imutils
import cv2
import os

In [4]:
videoFile = r'videos/testVideo.mp4'
configPath = r'YOLO_DB/yolov3.cfg'
weightsPath = r'YOLO_DB/yolov3.weights'
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

In [5]:
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
print(ln)

['yolo_82', 'yolo_94', 'yolo_106']


In [6]:
cam = cv2.VideoCapture(videoFile)
fps = int(cam.get(cv2.CAP_PROP_FPS))
print(fps)

25


In [7]:
LABELS = []
with open("YOLO_DB/coco.names", 'rt') as f:
    LABELS = f.read().rstrip("\n").split('\n')
print(len(LABELS))
print(LABELS.index("person"))

80
0


In [8]:
color_violation = (0,0, 255)
color_safe = (0,255,0)

In [9]:
while True:
    success, frame = cam.read()
    
    if not success:
        break
        
    frame = imutils.resize(frame, width=600)
    results = detectPeople(frame, net, ln, personIdx=LABELS.index('person'))
    
    
    voilation = set()
    
    if len(results) >=2:
        centroids = np.array([r[2] for r in results])
        distanceMap = dist.cdist(centroids, centroids, metric = "euclidean")
        
#   print(distanceMap.shape)
        for i in range(distanceMap.shape[0]):
            for j in range(i+1, distanceMap.shape[1]):
                
                #if distance between 2person is less minDistance
                if distanceMap[i, j] < MIN_DISTANCE:
                    voilation.add(i)
                    voilation.add(j)
        
    
    #Loop over results and make visualizations
    for (i, (prob, bbox, centroid)) in enumerate(results):
        (startX, startY, endX, endY)  = bbox
        (cX, cY) = centroid
        
        color = color_safe
        
        if i in voilation:
            color = color_violation
        
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        cv2.circle(frame, (cX, cY), 5, color, 1)
        
    
    
    cv2.imshow("Social Distance", frame)
        
    key = cv2.waitKey(10) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
    
    
    